In [997]:
import statsmodels.api as sm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [998]:
df = pd.read_csv('../data/details_stats.csv')
ballon_dor  = pd.read_csv('../data/BallonDOr_combined.csv')

In [999]:
ballon_dor['Year'] = ballon_dor['Year'].apply(lambda x: int(str(x)[-2:]))
ballon_dor = ballon_dor.query('Year >= 8 & Year <=23')
# ballon_dor['Share'] = np.log(ballon_dor['Share'])

df['Season'] = df['Season'].apply(lambda x: int(str(x)[-2:]))
df = df.query('Season >= 8 & Season <=23')

In [1000]:
stats23

Share  Champions League  Community Shield  \
Player         Year                                               
Alexis Sánchez 15    0.0114               3.0               0.0   
               15    0.0114               3.0               0.0   
               15    0.0114               3.0               0.0   
               15    0.0114               3.0               0.0   
               15    0.0114               3.0               0.0   
...                     ...               ...               ...   
Ángel Di María 14    0.0129               3.0               0.0   
Éric Abidal    11    0.0036               0.0               0.0   
İlkay Gündoğan 23    0.0054               1.0               0.0   
               23    0.0054               1.0               0.0   
               23    0.0054               1.0               0.0   

                     Coupe de la Ligue  DFB-Pokal  LaLiga  Leagues Cup  MLS  \
Player         Year                                                           
Alexis Sánchez 15                  0.0        0.0     0.0          0.0  0.0   
               15                  0.0        0.0     0.0          0.0  0.0   
               15                  0.0        0.0     0.0          0.0  0.0   
               15                  0.0        0.0     0.0          0.0  0.0   
               15                  0.0        0.0     0.0          0.0  0.0   
...                                ...        ...     ...          ...  ...   
Ángel Di María 14                  0.0        0.0     4.0          0.0  0.0   
Éric Abidal    11                  0.0        0.0     0.0          0.0  0.0   
İlkay Gündoğan 23                  0.0        0.0     0.0          0.0  0.0   
               23                  0.0        0.0     0.0          0.0  0.0   
               23                  0.0        0.0     0.0          0.0  0.0   

                     Supercopa  
Player         Year             
Alexis Sánchez 15          0.0  
               15          0.0  
               15          0.0  
               15          0.0  
               15          0.0  
...                        ...  
Ángel Di María 14          0.0  
Éric Abidal    11          0.0  
İlkay Gündoğan 23          0.0  
               23          0.0  
               23          0.0  

[1041 rows x 9 columns]

In [1001]:
bdo23 = ballon_dor.copy()
# bdo23 = ballon_dor.query('Year == 23')
bdo23 = bdo23[['Player', 'Share', 'Year']]

stats23 = df.copy()
stats23 = stats23.rename(columns={'Season' : 'Year'})
stats23 = stats23.replace('-', 0)

stats23 = pd.merge(stats23, bdo23, on=['Player', 'Year'], how='inner')

# stats23 = pd.get_dummies(stats23, columns=['Competition'])
stats23 = stats23.drop(columns=[ 'Club'])

stats23 = stats23.replace(np.nan, 0)
stats23 = stats23.query('Share != -inf')

stats23 = stats23.dropna(subset=stats23.select_dtypes(include=[float]).columns)
# stats23 = stats23.set_index(['Player', 'Year'])

# aggs = {i: 'sum' for i in stats23.columns if i not in  ['Share', 'Competition', 'Player', 'Year']}
# aggs = {'Total' : 'sum'}
# aggs['Share'] = 'mean'

for i in stats23.columns:
    if i not in ['Share', 'Competition', 'Player', 'Year']:
        stats23[i] = stats23[i].astype(float)

# stats23 = pd.get_dummies(stats23, columns=['Competition'])

# stats23 = stats23.groupby(['Player', 'Year']).agg(aggs)
# stats23 = stats23.astype(float)

# stats23.reset_index(inplace=True)

pd.pivot_table(stats23, index=['Player', 'Year', 'Competition'], aggfunc=np.sum)

/var/folders/7n/1v9m4ykn66ddbcxt5nwgg9280000gn/T/ipykernel_57279/3135805456.py:35: FutureWarning: The provided callable <function sum at 0x11ababe20> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(stats23, index=['Player', 'Year', 'Competition'], aggfunc=np.sum)


1 - 15  16 - 30  31 - 45  45+  \
Player         Year Competition                                           
Alexis Sánchez 15   Champions League         0.0      1.0      1.0  0.0   
                    Champions League Qu.     0.0      0.0      0.0  1.0   
                    EFL Cup                  1.0      0.0      0.0  0.0   
                    FA Cup                   0.0      0.0      1.0  0.0   
                    Premier League           1.0      3.0      3.0  1.0   
...                                          ...      ...      ...  ...   
Ángel Di María 14   LaLiga                   0.0      2.0      0.0  0.0   
Éric Abidal    11   Copa del Rey             0.0      0.0      0.0  0.0   
İlkay Gündoğan 23   Champions League         0.0      0.0      0.0  0.0   
                    FA Cup                   1.0      0.0      0.0  0.0   
                    Premier League           1.0      3.0      2.0  0.0   

                                          46 - 60  61 - 75  76 - 90  90+  \
Player         Year Competition                                            
Alexis Sánchez 15   Champions League          1.0      0.0      0.0  0.0   
                    Champions League Qu.      0.0      0.0      0.0  0.0   
                    EFL Cup                   0.0      0.0      0.0  0.0   
                    FA Cup                    1.0      0.0      1.0  0.0   
                    Premier League            1.0      4.0      1.0  2.0   
...                                           ...      ...      ...  ...   
Ángel Di María 14   LaLiga                    1.0      1.0      0.0  0.0   
Éric Abidal    11   Copa del Rey              0.0      1.0      0.0  0.0   
İlkay Gündoğan 23   Champions League          1.0      0.0      0.0  0.0   
                    FA Cup                    1.0      0.0      0.0  0.0   
                    Premier League            2.0      0.0      0.0  0.0   

                                          Extra time   Share  Total  
Player         Year Competition                                      
Alexis Sánchez 15   Champions League             0.0  0.0114    3.0  
                    Champions League Qu.         0.0  0.0114    1.0  
                    EFL Cup                      0.0  0.0114    1.0  
                    FA Cup                       1.0  0.0114    4.0  
                    Premier League               0.0  0.0114   16.0  
...                                              ...     ...    ...  
Ángel Di María 14   LaLiga                       0.0  0.0129    4.0  
Éric Abidal    11   Copa del Rey                 0.0  0.0036    1.0  
İlkay Gündoğan 23   Champions League             0.0  0.0054    1.0  
                    FA Cup                       0.0  0.0054    2.0  
                    Premier League               0.0  0.0054    8.0  

[1038 rows x 11 columns]

In [1002]:
stats23

,Year,Competition,1 - 15,16 - 30,31 - 45,45+,46 - 60,61 - 75,76 - 90,90+,Extra time,Total,Player,Share
0,22,Premier League,2.0,0.0,6.0,1.0,2.0,5.0,2.0,0.0,0.0,18.0,Cristiano Ronaldo,0.0000
1,22,Champions League,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,6.0,Cristiano Ronaldo,0.0000
2,21,Serie A,4.0,3.0,7.0,1.0,3.0,2.0,8.0,1.0,0.0,29.0,Cristiano Ronaldo,0.0654
3,21,Italy Cup,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,Cristiano Ronaldo,0.0654
4,21,Supercoppa Italiana,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,Cristiano Ronaldo,0.0654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,23,DFL-Supercup,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Jamal Musiala,0.0000
1037,23,Bundesliga,1.0,4.0,2.0,0.0,3.0,3.0,2.0,0.0,0.0,15.0,Randal Kolo Muani,0.0000
1038,23,DFB-Pokal,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,6.0,Randal Kolo Muani,0.0000
1039,23,Champions League,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,Randal Kolo Muani,0.0000


In [1003]:
pivot_stats23 = stats23[['Player', 'Year', 'Competition', 'Total', 'Share']].pivot_table(index=['Player', 'Year'], columns='Competition', aggfunc='sum')
pivot_stats23['Total'].replace(np.nan, 0, inplace=True)


/var/folders/7n/1v9m4ykn66ddbcxt5nwgg9280000gn/T/ipykernel_57279/3040409925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_stats23['Total'].replace(np.nan, 0, inplace=True)


In [1004]:
stats23 = pivot_stats23['Total'].reset_index().merge(stats23[['Player', 'Year', 'Share']], on=['Player', 'Year'], how='inner')
for column in stats23.columns:
    if column not in ['Player', 'Year', 'Share']:
        stats23[column] = stats23[column].astype(float)
stats23.set_index(['Player', 'Year'], inplace=True)

In [1005]:
stats23

Allianz Cup  Bundesliga  Champions League  \
Player         Year                                              
Alexis Sánchez 15            0.0         0.0               3.0   
               15            0.0         0.0               3.0   
               15            0.0         0.0               3.0   
               15            0.0         0.0               3.0   
               15            0.0         0.0               3.0   
...                          ...         ...               ...   
Ángel Di María 14            0.0         0.0               3.0   
Éric Abidal    11            0.0         0.0               0.0   
İlkay Gündoğan 23            0.0         0.0               1.0   
               23            0.0         0.0               1.0   
               23            0.0         0.0               1.0   

                     Champions League Qu.  Club World Cup  Community Shield  \
Player         Year                                                           
Alexis Sánchez 15                     1.0             0.0               0.0   
               15                     1.0             0.0               0.0   
               15                     1.0             0.0               0.0   
               15                     1.0             0.0               0.0   
               15                     1.0             0.0               0.0   
...                                   ...             ...               ...   
Ángel Di María 14                     0.0             0.0               0.0   
Éric Abidal    11                     0.0             0.0               0.0   
İlkay Gündoğan 23                     0.0             0.0               0.0   
               23                     0.0             0.0               0.0   
               23                     0.0             0.0               0.0   

                     Conference League  Copa del Rey  Copa do Brasil  \
Player         Year                                                    
Alexis Sánchez 15                  0.0           0.0             0.0   
               15                  0.0           0.0             0.0   
               15                  0.0           0.0             0.0   
               15                  0.0           0.0             0.0   
               15                  0.0           0.0             0.0   
...                                ...           ...             ...   
Ángel Di María 14                  0.0           4.0             0.0   
Éric Abidal    11                  0.0           1.0             0.0   
İlkay Gündoğan 23                  0.0           0.0             0.0   
               23                  0.0           0.0             0.0   
               23                  0.0           0.0             0.0   

                     Coupe de France  ...  Supercopa  Supercoppa Italiana  \
Player         Year                   ...                                   
Alexis Sánchez 15                0.0  ...        0.0                  0.0   
               15                0.0  ...        0.0                  0.0   
               15                0.0  ...        0.0                  0.0   
               15                0.0  ...        0.0                  0.0   
               15                0.0  ...        0.0                  0.0   
...                              ...  ...        ...                  ...   
Ángel Di María 14                0.0  ...        0.0                  0.0   
Éric Abidal    11                0.0  ...        0.0                  0.0   
İlkay Gündoğan 23                0.0  ...        0.0                  0.0   
               23                0.0  ...        0.0                  0.0   
               23                0.0  ...        0.0                  0.0   

                     Série A  Taça de Portugal  Trophée des Champions  \
Player         Year                                                     
Alexis Sánchez 15        0.0               0.0   

In [1006]:
glm1 =  sm.GLM(endog=stats23['Share'], exog=stats23.drop(columns=['Share']), family=sm.families.Gaussian())
stats23 = sm.add_constant(stats23)
results = glm1.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                 1041
Model:                            GLM   Df Residuals:                      998
Model Family:                Gaussian   Df Model:                           42
Link Function:               Identity   Scale:                       0.0035763
Method:                          IRLS   Log-Likelihood:                 1477.0
Date:                Fri, 15 Dec 2023   Deviance:                       3.5691
Time:                        16:39:30   Pearson chi2:                     3.57
No. Iterations:                     3   Pseudo R-squ. (CS):             0.7001
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Allianz Cup                  0.0084      0.033      0.256      0.798      -0.056       0.072
Bundesliga                  -0.0003      0.000     -0.516      0.606      -0.001       0.001
Champions League             0.0103      0.001     12.389      0.000       0.009       0.012
Champions League Qu.        -0.0096      0.006     -1.695      0.090      -0.021       0.001
Club World Cup              -0.0010      0.003     -0.312      0.755      -0.008       0.006
Community Shield            -0.0274      0.011     -2.566      0.010      -0.048      -0.006
Conference League           -0.0077      0.043     -0.178      0.859      -0.092       0.077
Copa del Rey                 0.0015      0.002      0.779      0.436      -0.002       0.005
Copa do Brasil               0.0317      0.060      0.530      0.596      -0.086       0.149
Coupe de France              0.0025      0.005      0.473      0.637      -0.008       0.013
Coupe de la Ligue           -0.0183      0.008     -2.336      0.019      -0.034      -0.003
DFB-Pokal                   -0.0057      0.003     -2.269      0.023      -0.011      -0.001
DFL-Supercup                -0.0084      0.008     -1.068      0.286      -0.024       0.007
EFL Cup                     -0.0024      0.003     -0.764      0.445      -0.008       0.004
Eredivisie                  -0.0018      0.001     -1.469      0.142      -0.004       0.001
Europa League               -0.0002      0.002     -0.143      0.886      -0.003       0.003
Europa League Qualifying     0.0018      0.005      0.359      0.720      -0.008       0.012
FA Cup                      -0.0046      0.003     -1.484      0.138      -0.011       0.001
Hessenliga                   0.0005      0.009      0.059      0.953      -0.016       0.017
Italy Cup                   -0.0035      0.005     -0.667      0.505      -0.014       0.007
KNVB beker                  -0.0026      0.008     -0.317      0.751      -0.019       0.014
LaLiga                       0.0021      0.000      6.344      0.000       0.001       0.003
Leagues Cup                  0.0270      0.004      7.406      0.000       0.020       0.034
Libertadores                -0.0020      0.023     -0.087      0.931      -0.047       0.043
Liga Portugal               -0.0030      0.002     -1.408      0.159      -0.007       0.001
Ligapokal                    0.0043      0.011      0.398      0.690      -0.017       0.026
Ligue 1                     -0.0006      0.001     -0.679      0.497      -0.002       0.001
MLS                          0.0027      0.000      7.406      0.000       0.002       0.003
National 2 - Grp. D          0.0070      0.017      0.420      0.675      -0.026       0.040
Oberliga Hessen              0.0028      0.030      0.094      0.925      -0.056       0.061
Premier League

In [1007]:
print(sorted(results.pvalues.items(), key=lambda x: x[1], reverse=False))

[('Champions League', 3.009548851969565e-35), ('MLS', 1.302255948812722e-13), ('Leagues Cup', 1.302255948813847e-13), ('LaLiga', 2.2348909667491576e-10), ('Supercopa', 3.618361481695436e-08), ('Community Shield', 0.010273636955223568), ('Coupe de la Ligue', 0.019487870678618718), ('DFB-Pokal', 0.023278263259661074), ('Premier Liga', 0.06985091165283526), ('Russian Super Cup', 0.07704612120824363), ('Russian Cup', 0.07704612120824375), ('Champions League Qu.', 0.09005554141862178), ('FA Cup', 0.13768965786265386), ('Eredivisie', 0.1416999801649574), ('Liga Portugal', 0.15914634384137705), ('Premier League', 0.18072469888861653), ('UEFA-Cup Qualifikation', 0.22135538796471976), ('Taça de Portugal', 0.2846356141497689), ('DFL-Supercup', 0.2855882775473889), ('UEFA Super Cup', 0.30120665231498855), ('Super League', 0.3454888182084249), ('Supercoppa Italiana', 0.35548222511852656), ('Copa del Rey', 0.43603810698978596), ('EFL Cup', 0.4450658267883789), ('Ligue 1', 0.49743912517489197), ('It

In [1008]:
leagues = []
for i in results.pvalues.items():
    if i[1] <= 0.05:
        leagues.append(i[0])

In [1009]:
train.isna().sum()

const                0
Share                0
Champions League     0
Community Shield     0
Coupe de la Ligue    0
DFB-Pokal            0
LaLiga               0
Leagues Cup          0
MLS                  0
Supercopa            0
dtype: int64

In [1010]:
stats23 = stats23[['Share'] + leagues]
glm1 =  sm.GLM(endog=stats23['Share'], exog=stats23.drop(columns=['Share']), family=sm.families.Gaussian())
stats23 = sm.add_constant(stats23)
results = glm1.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                 1041
Model:                            GLM   Df Residuals:                     1034
Model Family:                Gaussian   Df Model:                            6
Link Function:               Identity   Scale:                       0.0035922
Method:                          IRLS   Log-Likelihood:                 1456.3
Date:                Fri, 15 Dec 2023   Deviance:                       3.7144
Time:                        16:39:30   Pearson chi2:                     3.71
No. Iterations:                     3   Pseudo R-squ. (CS):             0.6863
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Champions League      0.0091      0.001     17.554      0.000       0.008       0.010
Community Shield     -0.0255      0.010     -2.439      0.015      -0.046      -0.005
Coupe de la Ligue    -0.0165      0.004     -3.763      0.000      -0.025      -0.008
DFB-Pokal            -0.0062      0.002     -3.591      0.000      -0.010      -0.003
LaLiga                0.0023      0.000     10.823      0.000       0.002       0.003
Leagues Cup           0.0275      0.003     10.316      0.000       0.022       0.033
MLS                   0.0027      0.000     10.316      0.000       0.002       0.003
Supercopa             0.0287      0.005      6.198      0.000       0.020       0.038
=====================================================================================
"""

In [1011]:
stats23.drop(columns=['const'], inplace=True)
train = stats23.query('Year < 21').copy()
test = stats23.query('Year == 21').copy()

In [1012]:
glm1 =  sm.GLM(exog=train.drop(columns=['Share']), endog=train['Share'], family=sm.families.Gaussian())
train = sm.add_constant(train)
results = glm1.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                  807
Model:                            GLM   Df Residuals:                      801
Model Family:                Gaussian   Df Model:                            5
Link Function:               Identity   Scale:                       0.0035838
Method:                          IRLS   Log-Likelihood:                 1130.2
Date:                Fri, 15 Dec 2023   Deviance:                       2.8706
Time:                        16:39:30   Pearson chi2:                     2.87
No. Iterations:                     3   Pseudo R-squ. (CS):             0.7228
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Champions League      0.0088      0.001     14.022      0.000       0.008       0.010
Community Shield     -0.0202      0.012     -1.634      0.102      -0.044       0.004
Coupe de la Ligue    -0.0159      0.004     -3.564      0.000      -0.025      -0.007
DFB-Pokal            -0.0051      0.002     -2.581      0.010      -0.009      -0.001
LaLiga                0.0022      0.000      9.395      0.000       0.002       0.003
Leagues Cup                0          0        nan        nan           0           0
MLS                        0          0        nan        nan           0           0
Supercopa             0.0457      0.005      8.691      0.000       0.035       0.056
=====================================================================================
"""

In [1013]:
# test.drop(columns=['const'], inplace=True)

In [1014]:
test = test.drop(columns=['Leagues Cup', 'MLS'])
# test = sm.add_constant(test)

In [1015]:
test

Share  Champions League  Community Shield  \
Player            Year                                               
Bruno Fernandes   21    0.0022               4.0               0.0   
                  21    0.0022               4.0               0.0   
                  21    0.0022               4.0               0.0   
                  21    0.0022               4.0               0.0   
Cesar Azpilicueta 21    0.0000               0.0               0.0   
...                        ...               ...               ...   
Romelu Lukaku     21    0.0096               4.0               0.0   
                  21    0.0096               4.0               0.0   
Rúben Dias        21    0.0004               0.0               0.0   
                  21    0.0004               0.0               0.0   
Simon Kjær        21    0.0029               0.0               0.0   

                        Coupe de la Ligue  DFB-Pokal  LaLiga  Supercopa  
Player            Year                                                   
Bruno Fernandes   21                  0.0        0.0     0.0        0.0  
                  21                  0.0        0.0     0.0        0.0  
                  21                  0.0        0.0     0.0        0.0  
                  21                  0.0        0.0     0.0        0.0  
Cesar Azpilicueta 21                  0.0        0.0     0.0        0.0  
...                                   ...        ...     ...        ...  
Romelu Lukaku     21                  0.0        0.0     0.0        0.0  
                  21                  0.0        0.0     0.0        0.0  
Rúben Dias        21                  0.0        0.0     0.0        0.0  
                  21                  0.0        0.0     0.0        0.0  
Simon Kjær        21                  0.0        0.0     0.0        0.0  

[74 rows x 7 columns]

In [1016]:
results.predict(test.drop(columns=['Share']))

ValueError: shapes (74,6) and (8,) not aligned: 6 (dim 1) != 8 (dim 0)

### OLD BASELINE

In [ ]:
glm2 =  sm.GLM(endog=stats23_2['Share'], exog=stats23_2.drop(columns=['Share']), family=sm.families.Gaussian())
results = glm2.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                  359
Model:                            GLM   Df Residuals:                      352
Model Family:                Gaussian   Df Model:                            6
Link Function:               Identity   Scale:                       0.0046930
Method:                          IRLS   Log-Likelihood:                 456.56
Date:                Fri, 15 Dec 2023   Deviance:                       1.6519
Time:                        16:33:37   Pearson chi2:                     1.65
No. Iterations:                     3   Pseudo R-squ. (CS):             0.3573
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0138      0.006     -2.267      0.023      -0.026      -0.002
16 - 30        0.0055      0.002      2.309      0.021       0.001       0.010
45+           -0.0145      0.007     -2.083      0.037      -0.028      -0.001
46 - 60       -0.0037      0.002     -1.613      0.107      -0.008       0.001
61 - 75       -0.0054      0.002     -2.463      0.014      -0.010      -0.001
Extra time     0.0308      0.011      2.842      0.004       0.010       0.052
Total          0.0035      0.001      4.043      0.000       0.002       0.005
==============================================================================
"""

In [ ]:
stats23_3 = stats23_2.copy()
stats23_3 = stats23_3.drop(columns=['46 - 60'])

In [ ]:
glm3 =  sm.GLM(endog=stats23_3['Share'], exog=stats23_3.drop(columns=['Share']), family=sm.families.Gaussian())
results = glm3.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                  359
Model:                            GLM   Df Residuals:                      353
Model Family:                Gaussian   Df Model:                            5
Link Function:               Identity   Scale:                       0.0047143
Method:                          IRLS   Log-Likelihood:                 455.24
Date:                Fri, 15 Dec 2023   Deviance:                       1.6641
Time:                        16:33:37   Pearson chi2:                     1.66
No. Iterations:                     3   Pseudo R-squ. (CS):             0.3513
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0141      0.006     -2.316      0.021      -0.026      -0.002
16 - 30        0.0066      0.002      2.880      0.004       0.002       0.011
45+           -0.0143      0.007     -2.040      0.041      -0.028      -0.001
61 - 75       -0.0046      0.002     -2.160      0.031      -0.009      -0.000
Extra time     0.0325      0.011      3.013      0.003       0.011       0.054
Total          0.0026      0.001      3.938      0.000       0.001       0.004
==============================================================================
"""